In [1]:
import pandas as pd
import os

In [2]:
import glob

def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).__next__()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files:
                if "Bibliography" in file:
                    continue
                if("doclist" in file):
                    continue
                if(".docx" in file):
                    r.append(os.path.join(subdir, file))                                                                         
    return r                                                                                                          


In [3]:
file_list = list_files("/Users/Mihir/Downloads/Sample/")

In [4]:
import docx
import pandas as pd
words = []
for i in file_list:
    doc = docx.Document(i)
    words.append([p.text for p in doc.paragraphs])
xml = "/Users/Mihir/Downloads/Sample/wsj.xlsx"
for i in range(len(words)):
    words[i] = ' '.join(words[i])
    words[i] = words[i][words[0].index("Body"):]

In [10]:
from pandas import DataFrame
df = DataFrame(words, columns = ['text'])
import re
# Remove punctuation
df['processed'] = df['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df['processed'] = df['text'].map(lambda x: x.lower())
# Print out the first rows of papers

In [11]:
print(words[100])

   The Washington Post March 18, 2016 Friday, Every Edition  
Copyright 2016 The Washington Post All Rights Reserved 
 Distribution: Every Zone Section: WEEKEND; Pg. T27 Length: 728 words Byline: Ann Hornaday Body   All of the tough ethical implications of modern warfare come to the fore in this tense play-by-play Helen Mirren brings gimlet-eyed, tungsten-spined intensity to her role as a ruthlessly calculating British army colonel in "Eye in the Sky," Gavin Hood's taut, well-constructed thriller. Unfolding almost in real time, this soberingly effective nail-biter follows the tactical, legal and ethical implications of a drone operation in East Africa that unexpectedly escalates from a surveillance job to a missile strike. And in Hood's capable hands, what could easily be a talky, theatrical chamber piece turns into a dynamic work of cinema. The characters and the tense, increasingly dire dialogue drive the action of "Eye in the Sky," with Mirren's Col. Powell overseeing operations fro

In [12]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = " ".join(sent)
        texts_out.append([token.lemma_ for token in doc])
    return texts_out

In [13]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = df.processed.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['body', 'the', 'feature', 'directing', 'debut', 'of', 'the', 'gifted', 'actor', 'peter', 'riegert', 'king', 'of', 'the', 'corner', 'comes', 'from', 'fiction', 'collection', 'titled', 'bad', 'jews', 'and', 'other', 'stories', 'by', 'gerald', 'shapiro', 'something', 'about']


In [14]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


In [15]:
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = " ".join(sent)
        texts_out.append([token.lemma_ for token in doc])
    return texts_out

In [16]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_trigrams(data_words_nostops)
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_trigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

In [17]:
import nltk
words = []
for i in range(len(data_words_bigrams)):
    wn = nltk.WordNetLemmatizer()
    words.append([wn.lemmatize(word) for word in data_words_bigrams[i]])

In [18]:
#for i in range(len(words)):
##    ne = nltk.tokenize.word_tokenize(words[i])
#    words[i] = [i for i in ne if i not in stopwords]
#    porter = PorterStemmer()
 #   words[i] = [porter.stem(word) for word in words[i]]

In [19]:
#df = pd.read_excel('/Users/Mihir/Downloads/Sample/wsj.xlsx', engine='openpyxl') 
#df = df.dropna()
#l = df['Content'].tolist()
#print(type(l[0]))
#print(type(words[0]))
#for x in l:
#    words.append(x)
    

In [20]:
import re
import string
import nltk
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem.porter import PorterStemmer

In [21]:
#for i in range(len(words)):
 #   words[i] = re.sub("[^-9A-Za-z ]", "" , words[i])
  #  words[i] = "".join([i.lower() for i in words[i] if i not in string.punctuation])
   # ne = nltk.tokenize.word_tokenize(words[i])
    #words[i] = [i for i in ne if i not in stopwords]
   # porter = PorterStemmer()
   # words[i] = [porter.stem(word) for word in words[i]]


In [28]:
from nltk.corpus import wordnet as wn
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}
verbs = {x.name().split('.', 1)[0] for x in wn.all_synsets('v')}

In [23]:
for i in words:
    if(type(i) == float):
        print(i)

In [24]:
from nltk.corpus import wordnet
dictionary = {}
for i in range(len(words)):
    s = set(words[i])
    for w in s:
        if(w in dictionary):
            dictionary[w]+=1
        else:
            dictionary[w] = 0
for i in range(len(words)):
    temp = []
    for w in words[i]:
        temp.append(w)
    for w in words[i]:
        if(dictionary[w]>=(len(words) * 0.85) or len(w)<=2 or (w not in nouns)):
            temp.remove(w)
    words[i] = temp

In [25]:
print(len(words))


1943


In [26]:
print(words[2])

['november', 'company', 'rock', 'review', 'animal', 'collective', 'set', 'sunday', 'night', 'webster', 'hall', 'rough', 'scraping', 'sound', 'loop', 'swelling', 'latest', 'run', 'great', 'new_york', 'art', 'rock', 'animal', 'collective', 'way', 'fixation', 'oceanic', 'expanse', 'youth', 'stasis', 'radio', 'obsessive', 'pastoral', 'animal', 'collective', 'brooklyn', 'album', 'hour', 'studio', 'sound', 'album', 'sung', 'tongs', 'feel', 'fat', 'cat', 'song', 'form', 'echo', 'loop', 'speed', 'voice', 'song', 'title', 'like', 'grass', 'bee', 'lyric', 'place', 'interaction', 'dream', 'world', 'nature', 'geography', 'song', 'individual', 'time', 'scale', 'nothing', 'three', 'minute', 'pop', 'expectation', 'lyric', 'say', 'piece', 'band', 'guitarist', 'tare', 'drift', 'stretch', 'sound', 'forest', 'city', 'block', 'summer', 'night', 'four', 'member', 'band', 'still', 'wilderness', 'geologist', 'voice', 'sound', 'music', 'music', 'core', 'chord', 'two', 'skiffle', 'like', 'drumbeat', 'bear', 'n

In [27]:
print(words[0])

['feature', 'debut', 'actor', 'peter', 'king', 'corner', 'fiction', 'collection', 'bad', 'jew', 'story', 'leo', 'sack', 'test', 'get', 'let', 'line', 'job', 'decade', 'life', 'viewer', 'leo', 'way', 'movie', 'minute', 'mine', 'film', 'spectacular', 'cast', 'depth', 'horizon', 'ending', 'leo', 'like', 'deputy', 'see', 'idea', 'young', 'sneak', 'leo', 'age', 'vulnerability', 'work', 'duty', 'new_york', 'one', 'visiting', 'father', 'southwest', 'regularity', 'home', 'life', 'given', 'share', 'house', 'wife', 'isabella', 'daughter', 'johnson', 'upset', 'first', 'curfew', 'horror', 'leo', 'self', 'old', 'pop', 'high', 'school', 'crush', 'interest', 'leo', 'compatibility', 'hotel', 'assignation', 'one', 'two', 'wince', 'episode', 'logorrhea', 'leo', 'drift', 'post', 'confrontation', 'woman', 'husband', 'get', 'punch', 'minute', 'film', 'moment', 'catharsis', 'familiar', 'face', 'cast', 'member', 'proceeding', 'leo', 'dad', 'codger', 'gal', 'offer', 'brand', 'authority', 'crop', 'rabbi', 'koo

In [30]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Mihir/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [31]:
for i in range(len(words)):
    wn = nltk.WordNetLemmatizer()
    words[i] = [wn.lemmatize(word) for word in words[i]]

In [32]:
print(words[0])

['feature', 'debut', 'actor', 'peter', 'king', 'corner', 'fiction', 'collection', 'bad', 'jew', 'story', 'leo', 'sack', 'test', 'get', 'let', 'line', 'job', 'decade', 'life', 'viewer', 'leo', 'way', 'movie', 'minute', 'mine', 'film', 'spectacular', 'cast', 'depth', 'horizon', 'ending', 'leo', 'like', 'deputy', 'see', 'idea', 'young', 'sneak', 'leo', 'age', 'vulnerability', 'work', 'duty', 'new_york', 'one', 'visiting', 'father', 'southwest', 'regularity', 'home', 'life', 'given', 'share', 'house', 'wife', 'isabella', 'daughter', 'johnson', 'upset', 'first', 'curfew', 'horror', 'leo', 'self', 'old', 'pop', 'high', 'school', 'crush', 'interest', 'leo', 'compatibility', 'hotel', 'assignation', 'one', 'two', 'wince', 'episode', 'logorrhea', 'leo', 'drift', 'post', 'confrontation', 'woman', 'husband', 'get', 'punch', 'minute', 'film', 'moment', 'catharsis', 'familiar', 'face', 'cast', 'member', 'proceeding', 'leo', 'dad', 'codger', 'gal', 'offer', 'brand', 'authority', 'crop', 'rabbi', 'koo

In [33]:
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora


In [34]:
id2word = corpora.Dictionary(words)
texts = words
corpus = [id2word.doc2bow(text) for text in words]

In [35]:
print(corpus[:1][0][:30])


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 4), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1)]


In [36]:
from pprint import pprint
# number of topics
num_topics = 6
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.009*"company" + 0.007*"official" + 0.007*"united_states" + 0.006*"say" + '
  '0.006*"aircraft" + 0.006*"one" + 0.006*"military" + 0.006*"year" + '
  '0.006*"group" + 0.006*"technology"'),
 (1,
  '0.010*"official" + 0.008*"strike" + 0.008*"military" + 0.008*"american" + '
  '0.007*"government" + 0.006*"one" + 0.006*"administration" + '
  '0.005*"pakistan" + 0.005*"people" + 0.005*"yemen"'),
 (2,
  '0.012*"official" + 0.009*"strike" + 0.009*"military" + 0.008*"aircraft" + '
  '0.008*"american" + 0.007*"one" + 0.006*"attack" + 0.006*"government" + '
  '0.005*"technology" + 0.005*"say"'),
 (3,
  '0.014*"military" + 0.009*"aircraft" + 0.009*"official" + 0.008*"pakistan" + '
  '0.008*"strike" + 0.007*"american" + 0.007*"one" + 0.007*"government" + '
  '0.006*"company" + 0.006*"united_states"'),
 (4,
  '0.014*"strike" + 0.012*"official" + 0.011*"american" + 0.010*"pakistan" + '
  '0.009*"military" + 0.009*"attack" + 0.008*"aircraft" + 0.007*"year" + '
  '0.007*"united_states" + 0.00

In [37]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('/Users/Mihir/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, '/Users/Mihir/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.013126 -0.001197       1        1  30.423289
0      0.014255  0.001856       2        1  19.566081
3     -0.002422  0.009561       3        1  18.611334
5      0.006041 -0.006245       4        1  15.509030
2     -0.001608  0.004074       5        1   9.530267
1     -0.003139 -0.008050       6        1   6.360000, topic_info=           Term         Freq        Total Category  logprob  loglift
388    official  4555.000000  4555.000000  Default  30.0000  30.0000
182    military  4196.000000  4196.000000  Default  29.0000  29.0000
623      strike  4183.000000  4183.000000  Default  28.0000  28.0000
337    american  3568.000000  3568.000000  Default  27.0000  27.0000
100         say  2205.000000  2205.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
149     company   127.824567  2830.525465   Topic6  -5.4360  -0.3424
873         may    96.255615  1505.329818   Topic6  -5.7196   0.0054
168       group   104.186922  1852.951374   Topic6  -5.6404  -0.1232
117         two   103.983061  2039.127316   Topic6  -5.6424  -0.2209
449  technology    97.749986  1875.315581   Topic6  -5.7042  -0.1990

[545 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
9751      2  0.264605       abkhazian
9751      5  0.264605       abkhazian
9953      4  0.514027     adjudicator
324       1  0.302528  administration
324       2  0.142518  administration
...     ...       ...             ...
4840      2  0.506913          zephyr
4840      3  0.253456          zephyr
4840      4  0.084485          zephyr
9008      4  0.396592         ziegler
9008      6  0.396592         ziegler

[1540 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 4, 6, 3, 2])

In [ ]:
df = LDAvis_prepared[1]
df = df.drop(columns = ["Freq", "Total", "logprob", "loglift"])
print(df)
compression_opts = dict(method='zip',
                         archive_name='out.csv')  
df.to_csv('out.zip', index=False,
           compression=compression_opts)  

In [27]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=words, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
from gensim.models import CoherenceModel


In [57]:
 import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()


 63%|██████▎   | 341/540 [2:45:38<1:36:39, 29.14s/it]A

 54%|█████▍    | 294/540 [1:37:48<1:05:42, 16.03s/it]


100%|██████████| 540/540 [2:49:29<00:00, 18.83s/it]


In [1]:
model_results['Topics']

NameError: name 'model_results' is not defined